# My Cryptocurrencies Portfolio

<br>This notebook displays a record of your deposits, a table with your current portfolio and a time series graph with the evolution of your portfolio.<br>
Currency data is fetched in real-time.<br>
To update your deposits or cryptocurrencies amount information, run the Update_Portfolio script

If / When graphs stop rendering, visit [your Plotly account](https://plot.ly/organize/home/) and delete all saved plots.

In [30]:
from IPython.display import HTML

In [31]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<i> <font size="2">The raw code for this IPython notebook is hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.<font><i>''') 

In [32]:
""" 
My Crypto Portfolio
Created on Wed Jan  3 17:13:29 2018
@author: marcel torretta
"""
import os
import time
from datetime import datetime, date, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import plotly
plotly.tools.set_credentials_file(plotly.tools.set_credentials_file(username='your_plotly_username', api_key='your_plotly_api_key'))
import plotly.plotly as py
import plotly.graph_objs as go
os.chdir("C:/Users/marce/Documents/6_DS_Projects/projects/1_Criptocurrencies_webscraping")

In [33]:
try:
    tseries = pd.read_csv("./tseries.csv", index_col = 0) 
except:
    tseries = pd.DataFrame(columns = ['ID', 'Name', 'Price_USD', 'Change_24h_%','My_Amount', 
                                     'My_Amount_USD', 'My_Amount_BRL', 'Date', 'Deposited_BRL'])

tseries['Date'] = pd.to_datetime(tseries['Date'])

In [34]:
#%% My Deposits
try:
    deposits_df = pd.read_csv("./my_deposits.csv", index_col = 0)
except:
    deposits_df = pd.DataFrame(columns = ['Date', 'Amount_BRL'])
    
deposits_df['Date'] = pd.to_datetime(deposits_df['Date'], format = '%Y-%m-%d')

## My Deposits

In [35]:
deposits_df

,Date,Amount_BRL
0,2017-10-18,1200.77
1,2017-10-19,2199.98
2,2017-10-20,2000.00
3,2017-10-29,500.02
4,2017-10-30,500.00
5,2017-10-31,2650.07


In [36]:
# My Cryptocurrency Amount
my_crypto_df = pd.read_csv("./my_crypto.csv", index_col = 0)
my_crypto_df['My_Amount'] = pd.to_numeric(my_crypto_df['My_Amount'])

In [37]:
# Extract currency table from website using pandas read_html
url = 'https://coinmarketcap.com/all/views/all/'
read_currency_table = pd.read_html(url)[0]

url2 = 'http://www.infomoney.com.br/mercados/cambio'

dollar = pd.read_html(url2)[0].iloc[0,3]
dollar = float(dollar) / 1000

currency_table = pd.read_html(url)[0]

In [38]:
# Splits 'Name' column from scraped table into 'ID' and 'Name'

currency_table.insert(1,'ID',0)
init = []
name = []
for n in currency_table['Name']:
    init.append(n.split()[0])
    name.append(n.split()[1])
currency_table['ID'] = init
currency_table['Name'] = name

In [39]:
#%% My currency table
# Subsetting currency table according to the currencies I possess, while also adding 'my_crypto' info.
my_currency_table = pd.merge(currency_table, my_crypto_df, on = 'ID')

In [40]:
#%% Select relevant columns
my_currency_table = my_currency_table[['ID', 'Name', 'Price', 'My_Amount']]

In [41]:
#%% Adjust columns names
# Some names have spaces on it, which is not ideal. 
# This part makes collumn names easier to handle
name = ''
col_names = []
for n in my_currency_table.columns.values.tolist():
    try: 
        name = "_".join(n.split())
    except:
        name = n
    name = name.replace('(','')
    name = name.replace(')','')
    if any(n in name for n in ['Price','Market','Volume','Supply']):
        name = name + '_USD' 
    if ('Change' in n):
        name = name + '_%'    
    col_names.append(name)
    
my_currency_table.columns = col_names

In [42]:
#%% Adjust column types
'''
Function to remove special characters from column values.
Transforming to numeric when possible
'''
def adjust_type(s): 
    try:
        s = s.replace('$','')
        s = s.replace('%','')
        s = pd.to_numeric(s)
    except:
        pass
    return(s)

In [43]:
'''
Apply function to whole dataframe, element-wise
'''
my_currency_table = my_currency_table.applymap(adjust_type)        

In [44]:
#%% My_amount in dollar
my_currency_table['My_Amount_USD'] = my_currency_table['My_Amount'] * my_currency_table['Price_USD']
my_currency_table['My_Amount_BRL'] = my_currency_table['My_Amount_USD'] * dollar

In [45]:
my_currency_table = my_currency_table.round({'Price_USD':2, 'My_Amount_USD':2, 'My_Amount_BRL':2})

##  My Portfolio

In [46]:
my_currency_table

,ID,Name,Price_USD,My_Amount,My_Amount_USD,My_Amount_BRL
0,BTC,Bitcoin,3617.93,0.273096,988.04,3679.47
1,ETH,Ethereum,122.60,0.149850,18.37,68.42
2,EOS,EOS,2.84,29.970000,85.11,316.97
3,ADA,Cardano,0.04,125.400000,5.16,19.23
4,MIOTA,IOTA,0.27,54.945000,14.91,55.52
5,WAVES,Waves,2.77,19.999960,55.40,206.31


In [47]:
#%% Yields 
# Adding 'Date' and 'Deposited' information in order to build time series
# The ts_day table ( Time Series - Day) compiles all relevant information of the day/time the script was run.

ts_day =  my_currency_table
ts_day['Date'] = pd.to_datetime(datetime.now())
ts_day['Deposited_BRL'] = deposits_df['Amount_BRL'].sum()

In [48]:
# The ts_day information will be added to the Time Series Table only if date is new, that is,
# different from the last date on tseries. This is to avoid cluttered points on the graph
try:
    if not (pd.to_datetime(datetime.now()).minute == tseries.loc[tseries.index[-1],'Date'].minute):
        tseries = pd.concat([tseries, ts_day], ignore_index = True, axis = 0, sort=True)[list(tseries.columns)]
except:
    tseries = pd.concat([tseries, ts_day], ignore_index = True, axis = 0, sort=True)[list(tseries.columns)]


In [49]:
tseries = tseries.round({'Price_USD':2, 'My_Amount_USD':2, 'My_Amount_BRL':2})

In [50]:
'''
Yields Time Series
Grouping table by date and getting sum of 'My_Amount_BRL of each group
'''
dates= []
amount_by_date = []
deposited_by_date = []

for name, group in tseries.groupby('Date', sort = False):
    dates.append(name)
    amount_by_date.append(group['My_Amount_BRL'].sum())
    deposited_by_date.append(group.loc[group.index[-1],'Deposited_BRL'])
    

In [51]:
d_total = go.Scatter(x = dates, y = amount_by_date, name = 'Total', mode = 'lines+markers')
d_deposited = go.Scatter(x = dates, y = deposited_by_date, line = dict(shape = 'hv', dash = 'dash'), name = 'Cummulative Deposits')

data = [d_total, d_deposited]

l = []
l_name = [] 
d = []

for i in set(tseries['ID']):
    l = list(tseries.loc[tseries['ID'] == i]['My_Amount_BRL'])
    d = list(tseries.loc[tseries['ID'] == i]['Date'])
    data.append(go.Scatter(x= d, y = l, name = i, opacity = 0.5, mode = 'lines+markers'))
    

In [52]:
layout = dict(
    title='My Cryptocurrencies Portfolio',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=7,
                     label='1w',
                     step='day',
                     stepmode='backward'),
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(count=1,
                    label='YTD',
                    step='year',
                    stepmode='todate'),
                dict(count=1,
                    label='1y',
                    step='year',
                    stepmode='backward'),
                dict(step='all')
            ])
        )   
    )
)


## Portfolio Time Series 

Plot is zoomable and interactive.

In [53]:
fig = dict( data = data, layout = layout)
py.iplot(fig)


In [54]:
tseries.to_csv("tseries.csv", float_format='%.8f')
